In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [4]:
train_features = pd.read_csv('dataset1_train_features.csv')
test_features = pd.read_csv('dataset1_test_features.csv')

train_types = []

for row in train_features['Type']:
    if row == 'Class':
        train_types.append(1)
    else:
        train_types.append(0)
        
train_features['Type_encode'] = train_types

test_types = []

for row in test_features['Type']:
    if row == 'Class':
        test_types.append(1)
    else:
        test_types.append(0)
        
test_features['Type_encode'] = test_types

X_train = train_features.loc[:, 'Ngram1_Entity':'Type_encode']
y_train = train_features['Match']

X_test = test_features.loc[:, 'Ngram1_Entity':'Type_encode']
y_test = test_features['Match']

df_train = train_features.loc[:, 'Ngram1_Entity':'Type_encode']
df_train['Match'] = train_features['Match']

df_test = test_features.loc[:, 'Ngram1_Entity':'Type_encode']
df_test['Match'] = test_features['Match']

X_train = X_train.fillna(value=0)
X_test = X_test.fillna(value=0)

In [21]:
# Create regularization penalty space
penaltys = ['l1', 'l2']

# Create regularization hyperparameter space
Cs = np.logspace(0, 4, 10)

class_weights = ['balanced', None]

In [22]:
data = []

for C in Cs:
    for penalty in penaltys:
        for class_weight in class_weights:
            LR = LogisticRegression(penalty=penalty, C=C, class_weight = class_weight)
            LR.fit(X_train, y_train)
            y_pred = LR.predict(X_test)
            y_prob = LR.predict_proba(X_test)
            print('C=', C, ' penalty=', penalty, 'class_weight=', class_weight)
            f1 = f1_score(y_test, y_pred)
            log = log_loss(y_test, y_prob)
            data.append((C, penalty, class_weight, f1_score, log))
            #print(classification_report(y_test, y_pred))
            break
            
dataset = pd.DataFrame(data, columns=['C', 'penalty', 'class_weight', 'f1-score', 'binary_crossentropy'])

dataset.to_csv('dataset1_logreg.csv', index=False)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


KeyboardInterrupt: 